In [540]:
from collections import OrderedDict
from noaa_coops import Station
from deepdiff import DeepDiff
import datetime as dt
import xarray as xr
import pandas as pd
import numpy as np
import calendar
import os
import json
import plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [2]:
station = Station(id='8723214')

In [3]:
def _format_date(datestr):
    dtdt = pd.to_datetime(datestr)
    return dt.datetime.strftime(dtdt, '%Y%m%d')

In [56]:
# desired_variables = [('Air Temperature', 'air_temperature'),
#                      ('Barometric Pressure', 'air_pressure'),
#                      ('Wind', 'wind'),
#                      ('Water Temperature', 'water_temperature'),
#                      ('Verified 6-Minute Water Level', 'water_level')]

desired_variables = {'Air Temperature':
                         {'product': 'air_temperature',
                          'vars': ['obs', 'flag']},
                    # 'Barometric Pressure':
                    #      {'product': 'air_pressure',
                    #       'vars': ['obs', 'flag']},
                    # 'Wind':
                    #      {'product': 'wind',
                    #       'vars': ['windspeed', 'winddir_deg', 'winddir', 'windgust', 'flag']},
                    # 'Water Temperature':
                    #      {'product': 'water_temperature',
                    #       'vars': ['obs', 'flag']},
                    'Verified 6-Minute Water Level':
                         {'product': 'water_level',
                          'vars': ['obs', 's', 'flag', 'wlevel_qc']}}

In [57]:
start_date = _format_date(pd.to_datetime('2023-04-26'))
end_date = _format_date(pd.to_datetime('today'))

print(start_date)
print(end_date)

unit_system = 'english'
datum = 'MHHW'
tz = 'lst'

20230426
20240425


In [58]:
for variable in list(desired_variables.keys()):
    print(f'Retrieving {variable.lower()} data')
    data = station.get_data(
            begin_date=start_date,
            end_date=end_date,
            product=desired_variables[variable]['product'],
            units=unit_system,
            datum=datum,
            time_zone=tz)
    data.columns = desired_variables[variable]['vars']
    data.index.name = 'Date'
    data['flag'] = data['flag'].str.split(',', expand=True).astype(int).sum(axis=1)
    
    if variable == 'wind':
        continue
    else
        data.loc[data['flag'] > 0, 'obs'] = np.nan

    
    
    # desired_variables[variable]['data'] = xr.DataArray(data)

Retrieving air temperature data
Retrieving verified 6-minute water level data


In [60]:
data

,obs,s,flag,wlevel_qc
Date,,,,
2023-04-26 00:00:00,0.082,0.013,0,v
2023-04-26 00:06:00,0.102,0.013,0,v
2023-04-26 00:12:00,0.125,0.020,0,v
2023-04-26 00:18:00,0.131,0.013,0,v
2023-04-26 00:24:00,0.157,0.020,0,v
...,...,...,...,...
2024-04-24 21:36:00,NaN,0.023,1,p
2024-04-24 21:42:00,NaN,0.023,1,p
2024-04-24 21:48:00,0.568,0.023,0,p


In [51]:
desired_variables['Verified 6-Minute Water Level']['data']

<xarray.DataArray (Date: 87577, dim_1: 4)> Size: 3MB
array([[0.082, 0.013, '0,0,0,0', 'v'],
       [0.102, 0.013, '0,0,0,0', 'v'],
       [0.125, 0.02, '0,0,0,0', 'v'],
       ...,
       [0.561, 0.023, '1,0,0,0', 'p'],
       [0.564, 0.023, '1,0,0,0', 'p'],
       [0.568, 0.023, '1,0,0,0', 'p']], dtype=object)
Coordinates:
  * Date     (Date) datetime64[ns] 701kB 2023-04-26 ... 2024-04-24T21:36:00
  * dim_1    (dim_1) object 32B 'wlevel' 's' 'wlevel_flag' 'wlevel_qc'

In [ ]:
xr.Dataset()

In [4]:
class Data:
    def __init__(self, stationname: str, stationid: str, units='metric', timezone='gmt', datum='MHHW',
                 outdir=None, hr_threshold=3, day_threshold=2, redownload=False, verbose=True) -> None:
        """Data class for downloading, formatting, and saving to file historical
        atmospheric (air temperature, barometric pressure, wind) and oceanographic
        (water temperature, water level) data from NOAA CO-OPS coastal tide stations.
        WARNING: This may take a while to initiate depending on the amount of data
        to be retrieved.
        
        Inputs:
            stationname: str, desired name of station. Does not need to be the CO-OPS
                name; it is used only for saving data to file.
            stationid: str, NOAA CO-OPS tide station number from which to retrieve data
            outdir: str, directory to save data to. Defaults to present working directory.
            units: str, either 'metric' or 'english', indicating the units to download
                data in. Defaults to 'metric'.
            timezone: str, one of either 'gmt' for Greenwich Mean Time, 'lst' for local
                standard time, or 'lst_ldt' for adjusted local standard/daylight savings
                time. Defaults to 'gmt'.
            datum: str, tidal datum for water level data. Options: 'STND', 'MHHW', 'MHW',
                'MTL', 'MSL', 'MLW', 'MLLW', 'NAVD' Defaults to 'MHHW'.
            hr_threshold: int, maximum number of hours of data that can be missing in a
                given day in order for that day to be included in the historical record.
                Default is 3.
            day_threshold: int, maximum number of days of data that can be missing in a
                given month in order for that month to be included in the historical record.
                Default is 2.
            redownload: Bool, if True, historical data will be redownloaded and the class
                instance will be re-initiated. Defaults to False.
            verbose: Bool, print statuses to screen. Defaults to True.
        """
        
        self.name = stationname
        self.dirname = self._camel(stationname)
        self.id = stationid
        self.unit_system = units.lower()
        self.tz = timezone.lower()
        self.datum = datum.upper()
        self.hr_threshold = hr_threshold
        self.day_threshold = day_threshold
        self.verbose = verbose
        self.variables = []
        today = self._format_date(pd.to_datetime('today'))
        
        # Check for valid arguments
        self._valid_units(self.unit_system)
        self._valid_tz(self.tz)
        self._valid_datum(self.datum)
        
        # Set data directory, creating station subdirectory if needed
        if outdir:
            self.outdir = os.path.join(outdir, self.dirname)
        else:
            self.outdir = os.path.join(os.getcwd(), self.dirname)

        # ============================================================
        # If 'redownload' argument is True OR if the directory station
        # name subdirectory does not exist within 'outdir', then create
        # that subdirectory and download historical data.
        if not os.path.exists(self.outdir) or redownload:
            if not os.path.exists(self.outdir):
                if self.verbose:
                    print('Creating new directory for this station.')
                os.makedirs(self.outdir)
        
            # Download all data and save to file
            self.download_data(start_date=None, end_date=None)
            # self.download_data(start_date=self._format_date('2023-01-01'), end_date=self._format_date('2023-10-01'))
            self.data.to_csv(os.path.join(self.outdir, 'observational_data_record.csv.gz'),
                             compression='infer')
            if self.verbose:
                print("Observational data written to file "\
                      f"'{os.path.join(self.outdir, 'observational_data_record.csv')}'.")

            # Store units
            self.unit_options = dict({
                'Air Temperature': {'metric': 'C', 'english': 'F'},
                'Barometric Pressure': {'metric': 'mb', 'english': 'mb'},
                'Wind Speed': {'metric': 'm/s', 'english': 'kn'},
                'Wind Gust': {'metric': 'm/s', 'english': 'kn'},
                'Wind Direction': {'metric': 'deg', 'english': 'deg'},
                'Water Temperature': {'metric': 'C', 'english': 'F'},
                'Water Level': {'metric': 'm', 'english': 'ft'}
            })
            self.units =  {k:v[self.unit_system] for k,v in self.unit_options.items() if k in self.variables}
            
            # Save class variables
            meta = dict({
                'stationname': self.name,
                'stationid': self.id,
                'outdir': self.outdir,
                'unit_system': self.unit_system,
                'tz': self.tz,
                'datum': self.datum,
                'hr_threshold': self.hr_threshold,
                'day_threshold': self.day_threshold,
                'variables': self.variables,
                'units': self.units})
            with open(os.path.join(self.outdir, 'metadata.json'), 'w') as fp:
                json.dump(meta, fp) 
                    
            # Create and save statistics dictionaries
            self.filtered_data = {var: self._filter_data(self.data[var],
                                                         hr_threshold=self.hr_threshold,
                                                         day_threshold=self.day_threshold) \
                                  for var in self.variables}
            # Daily stats
            self.daily_stats_dict = self.daily_stats()
            statsOutFile = os.path.join(self.outdir, 'statistics-daily.json')
            with open(statsOutFile, 'w') as fp:
                json.dump(self.daily_stats_dict, fp)
            if self.verbose:
                print(f"Observational daily statistics written to '{statsOutFile}'")
            # Monthly stats
            self.monthly_stats_dict = self.monthly_stats()
            statsOutFile = os.path.join(self.outdir, 'statistics-monthly.json')
            with open(statsOutFile, 'w') as fp:
                json.dump(self.monthly_stats_dict, fp)
            if self.verbose:
                print(f"Observational monthly statistics written to '{statsOutFile}'")            

        # ===================================================    
        # If historical data for this station already exists:
        else:
            # Load the metadata from file
            if self.verbose:
                print('Loading metadata from file')
            with open(os.path.join(self.outdir, 'metadata.json')) as m:
                meta = json.load(m)
            self._load_from_json(meta)
            
            # Load the historical data from file
            if self.verbose:
                print('Loading historical data from file')
            self.data = pd.read_csv(os.path.join(self.outdir, 'observational_data_record.csv.gz'),
                                    index_col=f'time_{self.tz}', parse_dates=True,
                                    compression='infer')
            
            # Load daily statistics from file
            if self.verbose:
                print('Loading daily statistics from file')
            with open(os.path.join(self.outdir, 'statistics-daily.json')) as s:
                self.daily_stats_dict = json.load(s)
            self.daily_stats_tables = self.stats_table(self.daily_stats_dict)

            # Load monthly statistics from file
            if self.verbose:
                print('Loading monthly statistics from file')
            with open(os.path.join(self.outdir, 'statistics-monthly.json')) as s:
                self.monthly_stats_dict = json.load(s)
            self.monthly_stats_tables = self.stats_table(self.monthly_stats_dict)

            # Clean and format
            self.filtered_data = {var: self._filter_data(self.data[var],
                                                         hr_threshold=self.hr_threshold,
                                                         day_threshold=self.day_threshold) \
                                  for var in self.variables}                
        if self.verbose:
            print('Done!')
        
    def download_data(self, start_date=None, end_date=None):
        """Download data from NOAA CO-OPS"""
        if self.verbose:
            print('Downloading historic data')
        
        # NOAA CO-OPS API
        self.station = Station(id=self.id)

        # List of data variables to combine at the end
        datasets = []

        # If no 'end_date' is passed, download through current day
        if not end_date:
            end_date = self._format_date(pd.to_datetime('today'))

        # Air temperature
        if 'Air Temperature' in self.station.data_inventory:
            self.variables.append('Air Temperature')
            if not start_date:
                start_date = self._format_date(self.station.data_inventory['Air Temperature']['start_date'])
            self._load_atemp(start_date=start_date, end_date=end_date)
            self.air_temp['atemp_flag'] = self.air_temp['atemp_flag'].str.split(',', expand=True).astype(int).sum(axis=1)
            self.air_temp.loc[self.air_temp['atemp_flag'] > 0, 'atemp'] = np.nan
            datasets.append(self.air_temp['atemp'])

        # # Barometric pressure
        # if 'Barometric Pressure' in self.station.data_inventory:
        #     self.variables.append('Barometric Pressure')
        #     if not start_date:
        #         start_date = self._format_date(self.station.data_inventory['Barometric Pressure']['start_date'])
        #     self._load_atm_pres(start_date=start_date, end_date=end_date)
            # self.pressure['apres_flag'] = self.pressure['apres_flag'].str.split(',', expand=True).astype(int).sum(axis=1)
            # self.pressure.loc[self.pressure['apres_flag'] > 0, 'apres'] = np.nan
        #     datasets.append(self.pressure['apres'])

        # # Wind
        # if 'Wind' in self.station.data_inventory:
        #     self.variables.extend(['Wind Speed', 'Wind Gust'])
        #     if not start_date:
        #         start_date = self._format_date(self.station.data_inventory['Wind']['start_date'])
        #     self._load_wind(start_date=start_date, end_date=end_date)
            # self.wind['windflag'] = self.wind['wind_flag'].str.split(',', expand=True).astype(int).sum(axis=1)
            # self.wind.loc[self.wind['wind_flag'] > 0, ['windspeed', 'windgust']] = np.nan
        #     datasets.append(self.wind[['windspeed', 'windgust']])

        # Water temperature
        if 'Water Temperature' in self.station.data_inventory:
            self.variables.append('Water Temperature')
            if not start_date:
                start_date = self._format_date(self.station.data_inventory['Water Temperature']['start_date'])
            self._load_water_temp(start_date=start_date, end_date=end_date)
            self.water_temp['wtemp_flag'] = self.water_temp['wtemp_flag'].str.split(',', expand=True).astype(int).sum(axis=1)
            self.water_temp.loc[self.water_temp['wtemp_flag'] > 0, 'wtemp'] = np.nan
            datasets.append(self.water_temp['wtemp'])

        # # Water level (tides)
        # if 'Verified 6-Minute Water Level' in self.station.data_inventory:
        #     self.variables.append('Water Level')
        #     if not start_date:
        #         start_date = self._format_date(self.station.data_inventory['Verified 6-Minute Water Level']['start_date'])
        #     self._load_water_level(start_date=start_date, end_date=end_date)
            # self.water_levels['wlevel_flag'] = self.water_levels['wlevel_flag'].str.split(',', expand=True).astype(int).sum(axis=1)
            # self.water_levels.loc[self.water_levels['wlevel_flag'] > 0, 'wlevel'] = np.nan
        #     datasets.append(self.water_levels['wlevel'])

        # Merge into single dataframe
        if self.verbose:
            print('Compiling data')
        self.data = pd.concat(datasets, axis=1)
        self.data.index.name = f'time_{self.tz}'
        self.data.columns = [i for i in self.variables]

    def update_data(self, start_date=None, end_date=None):
        """Download data from NOAA CO-OPS"""
        if self.verbose:
            print('Downloading latest data')

        # NOAA CO-OPS API
        self.station = Station(id=self.id)

        # List of data variables to combine at the end
        datasets = []
        
        # If no 'start_date' is passed, pick up from the last observation time
        if not start_date:
            start_date = self._format_date(self.data.index.max())
            
        # If no 'end_date' is passed, download through end of current date
        if not end_date:
            end_date = self._format_date(pd.to_datetime('today') + pd.Timedelta(days=1))
        
        # Air temperature
        if 'Air Temperature' in self.variables:
            self._load_atemp(start_date=start_date, end_date=end_date)
            datasets.append(self.air_temp['atemp'])

        # Barometric pressure
        if 'Barometric Pressure' in self.variables:
            self._load_atm_pres(start_date=start_date, end_date=end_date)
            datasets.append(self.pressure['apres'])

        # Wind
        if 'Wind Speed' in self.variables:
            self._load_wind(start_date=start_date, end_date=end_date)
            datasets.append(self.wind[['windspeed', 'windgust']])

        # Water temperature
        if 'Water Temperature' in self.variables:
            self._load_water_temp(start_date=start_date, end_date=end_date)
            datasets.append(self.water_temp['wtemp'])

        # Water level (tides)
        if 'Verified 6-Minute Water Level' in self.variables:
            self._load_water_level(start_date=start_date, end_date=end_date)
            datasets.append(self.water_levels['wlevel'])

        # Merge into single dataframe
        data = pd.concat(datasets, axis=1)
        if sum(~data.index.isin(self.data.index)) == 0:
            print('No new data available.')
        else:
            data.index.name = f'time_{self.tz}'
            data.columns = [i for i in self.variables]
            data = pd.concat([self.data,
                              data[data.index.isin(self.data.index) == False]],
                             axis=0)
            self.data = data
            self.filtered_data = {var:self._filter_data(self.data[var],
                                                        hr_threshold=self.hr_threshold,
                                                        day_threshold=self.day_threshold).sort_values(['YearDay']) \
                                 for var in self.variables}
            self.data.to_csv(os.path.join(self.outdir, 'observational_data_record.csv.gz'),
                             compression='infer')
            if self.verbose:
                print("Updated observational data written to file "\
                      f"'{os.path.join(self.outdir, 'observational_data_record.csv')}'.")
                print("Done! (Don't forget to run Data.update_stats() to update statistics.)")
    
    def update_stats(self):    
        """Calculate new statistics and update if any changes"""
        # Daily stats
        _new_daily_stats = self.daily_stats()
        if self._ordered(_new_daily_stats) != self._ordered(self.daily_stats_dict):
            if self.verbose:
                print('Daily stats dicts differ. Updating and saving to file.\n')
                print('*'*10)
                print('NEW RECORD')
                self._compare(self.daily_stats_dict, _new_daily_stats)
                print('*'*10)
            self.daily_stats_dict = _new_daily_stats
            # Write to file
            statsOutFile = os.path.join(self.outdir, 'statistics-daily.json')
            with open(statsOutFile, 'w') as fp:
                json.dump(self.daily_stats_dict, fp)
            if self.verbose:
                print(f"\nUpdated daily observational statistics written to '{statsOutFile}'")
        else:
            if self.verbose:
                print("No new daily records set.")

        # Monthly stats
        _new_monthly_stats = self.monthly_stats()
        if self._ordered(_new_monthly_stats) != self._ordered(self.monthly_stats_dict):
            if self.verbose:
                print('Monthly stats dicts differ. Updating and saving to file.\n')
                print('*'*10)
                print('NEW RECORD')
                self._compare(self.monthly_stats_dict, _new_monthly_stats)
                print('*'*10)
            self.monthly_stats_dict = _new_monthly_stats
            # Write to file
            statsOutFile = os.path.join(self.outdir, 'statistics-monthly.json')
            with open(statsOutFile, 'w') as fp:
                json.dump(self.monthly_stats_dict, fp)
            if self.verbose:
                print(f"\nUpdated daily observational statistics written to '{statsOutFile}'")
        else:
            if self.verbose:
                print("No new monthly records set.")

    def _ordered(self, obj):
        if isinstance(obj, dict):
            return sorted((k, self._ordered(v)) for k, v in obj.items())
        if isinstance(obj, list):
            return sorted(self._ordered(x) for x in obj)
        else:
            return obj

    def _format_date(self, datestr):
        dtdt = pd.to_datetime(datestr)
        return dt.datetime.strftime(dtdt, '%Y%m%d')
    
    def _camel(self, text):
        """Convert to camel case"""
        s = text.replace(',','').replace("-", " ").replace("_", " ")
        s = s.split()
        if len(text) == 0:
            return text
        return s[0].lower() + ''.join(i.capitalize() for i in s[1:])

    def _valid_units(self, unit):
        valid = {'metric', 'english'}
        if unit.lower() not in valid:
            raise ValueError("units: units must be one of %r." % valid)
    
    def _valid_tz(self, tz):
        valid = {'gmt', 'lst', 'lst_ldt'}
        if tz.lower() not in valid:
            raise ValueError("timezone: timezone must be one of %r." % valid)

    def _valid_datum(self, datum):
        valid = {'STND', 'MHHW', 'MHW', 'MTL', 'MSL', 'MLW', 'MLLW', 'NAVD'}
        if datum.upper() not in valid:
            raise ValueError("datum: datum must be one of %r." % valid)

    def _load_from_json(self, blob):
        for k, v in blob.items():
            setattr(self, k, v)
    
    def get_data(self):
        return self.data
        
    def _load_atemp(self, start_date, end_date):
        """Download air temperature data from NOAA CO-OPS
        from 'start_date' through current day.
        """
        if self.verbose:
            print('Retrieving air temperature data')
        self.air_temp = self.station.get_data(
            begin_date=start_date,
            end_date=end_date,
            product='air_temperature',
            units=self.unit_system,
            time_zone=self.tz)
        self.air_temp.columns = ['atemp', 'atemp_flag']
    
    def _load_wind(self, start_date, end_date):
        """Download wind data from NOAA CO-OPS
        from 'start_date' through current day.
        """
        if self.verbose:
            print('Retrieving wind data')
        self.wind = self.station.get_data(
            begin_date=start_date,
            end_date=end_date,
            product='wind',
            units=self.unit_system,
            time_zone=self.tz)
        self.wind.columns = ['windspeed', 'winddir_deg', 'winddir', 'windgust', 'wind_flag']
    
    def _load_atm_pres(self, start_date, end_date):
        """Download barometric pressure data from NOAA CO-OPS
        from 'start_date' through current day.
        """
        if self.verbose:
            print('Retrieving barometric pressure data')
        self.pressure = self.station.get_data(
            begin_date=start_date,
            end_date=end_date,
            product='air_pressure',
            units=self.unit_system,
            time_zone=self.tz)
        self.pressure.columns = ['apres', 'apres_flag']
    
    def _load_water_temp(self, start_date, end_date):
        """Download water temperature data from NOAA CO-OPS
        from 'start_date' through current day.
        """
        if self.verbose:
            print('Retrieving water temperature data')
        self.water_temp = self.station.get_data(
            begin_date=start_date,
            end_date=end_date,
            product='water_temperature',
            units=self.unit_system,
            time_zone=self.tz)
        self.water_temp.columns = ['wtemp', 'wtemp_flag']

    def _load_water_level(self, start_date, end_date):
        """Download water level tide data from NOAA CO-OPS
        from 'start_date' through current day.
        """
        if self.verbose:
            print('Retrieving water level tide data')
        self.water_levels = self.station.get_data(
            begin_date=start_date,
            end_date=end_date,
            product='water_level',
            datum=self.datum,
            units=self.unit_system,
            time_zone=self.tz)
        self.water_levels.columns = ['wlevel', 's', 'wlevel_flag', 'wlevel_qc']

    def _DOY(self, df):
        """Calculate year day out of 366"""
        import calendar
        # Day of year as integer
        df['YearDay'] = df.index.day_of_year.astype(int)
        # Years that are NOT leap years
        leapInd = [not calendar.isleap(i) for i in df.index.year]
        mask = (leapInd) & (df['Month'] > 2)
        # Advance by one day everything after February 28 
        df.loc[mask, 'YearDay'] += 1
        # Day of year for plotting
        # df['DOY'] = pd.to_datetime(df['YearDay'].astype(str), format="%j")
        df['DOY'] = pd.to_datetime(df.index.day_of_year.astype(str), format="%j")
        return df

    def _set_dayflag(self, data, threshold_hrs=3):
        """Set flag indicating whether each day contains at least
        (24 - 'threshold_hours') hours of data: True if yes, False if not.
        """
        def _flag(group, threshold=threshold_hrs):
            # gap = sum(group.index.to_series().diff() > pd.Timedelta(hours=1))
            gap = group.resample('1h').mean(numeric_only=True).isna().sum()
            return gap <= threshold

        newColName = data.name + '_DayFlag'
        data = pd.DataFrame(data[:])
        data[newColName] = data.groupby(pd.Grouper(freq='D')).agg(lambda x: _flag(group=x, threshold=threshold_hrs))
        data[newColName] = data.resample('D')[newColName].ffill()
        data = data.astype({newColName: 'bool'})
        return data
    
    def _set_monthflag(self, data, threshold_days=2):
        """Set flag indicating whether each month contains at least
        (n -'threshold_days') days of data, where n is the number of
        days in the given month: True if yes, False if not.
        """
        def _flag(group, threshold=threshold_days):
            try:
                # gap = sum(group.index.to_series().diff() > pd.Timedelta(days=1))
                # flag = gap <= threshold
                daysWithData = group.resample('1D').mean(numeric_only=True).dropna().size
                daysNoData =  pd.Period(group.index[0].strftime(format='%Y-%m-%d')).days_in_month - daysWithData
                flag = daysNoData <= threshold
                return pd.DataFrame(pd.Series(flag, index=group.index))
            except IndexError:
                return None

        origColName = data.name
        newColName = data.name + '_MonthFlag'
        data = pd.DataFrame(data[:])
        data['Year'] = data.index.year
        data['Month'] = data.index.month
        data['Day'] = data.index.day
        data[newColName] = data.groupby(['Year', 'Month'], group_keys=False)\
                               .apply(lambda x: _flag(x, threshold=threshold_days))
        return data

    def _filter_data(self, series, hr_threshold=3, day_threshold=2):
        """Removes days with more than 'hr_threshold' of missing data
        and months with more than 'day_threshold' days of missing data.
        """
        # Variable to process
        var = series.name

        # Remove days with >hr_threshold hours of missing data
        df_dayFlagged = self._set_dayflag(series[:], threshold_hrs=hr_threshold)
        dfss = df_dayFlagged.loc[df_dayFlagged[f'{var}_DayFlag'],:]

        # Remove months with >day_threshold days of missing data
        df_monFlagged = self._set_monthflag(dfss[var], threshold_days=day_threshold)
        dfss = df_monFlagged.loc[df_monFlagged[f'{var}_MonthFlag'],:].copy()
        
        # Add Year Day
        dfss = self._DOY(dfss)

        # Drop month flag
        dfss.drop(f'{var}_MonthFlag', axis=1, inplace=True)
        return dfss.sort_index()
            
    def daily_highs(self, var, decimals=1):
        """Daily highs for variable 'var'."""
        return self.filtered_data[var].groupby(pd.Grouper(freq='D')).max(numeric_only=True)
    
    def daily_lows(self, var, decimals=1):
        """Daily lows for variable 'var'."""
        return self.filtered_data[var].groupby(pd.Grouper(freq='D')).min(numeric_only=True)

    def daily_avgs(self, var, decimals=1, true_average=False):
        """Daily averages by calendar day for variable 'var' rounded to 'decimals'.
        If 'true_average' is True, all measurements from each 24-hour day will be
        used to calculate the average. Otherwise, only the maximum and minimum
        observations are used. Defaults to False.
        """
        if true_average:
            return self.filtered_data[var].groupby(pd.Grouper(freq='D')).mean(numeric_only=True).round(decimals)
        else:
            dailyHighs = self.daily_highs(var=var, decimals=decimals)
            dailyLows = self.daily_lows(var=var, decimals=decimals)
            return ((dailyHighs + dailyLows)/2).round(decimals)

    def daily_avg(self, var, decimals=1):
        """Daily averages for variable 'var' rounded to 'decimals'."""
        dailyAvgs = self.daily_avgs(var=var, decimals=decimals)
        dailyAvg = dailyAvgs.groupby('YearDay').mean(numeric_only=True).round(decimals)[var]
        dailyAvg.index = dailyAvg.index.astype(int)
        return dailyAvg
    
    def monthly_avg(self, var, decimals=1):
        """Monthly averages for variable 'var' rounded to 'decimals'."""
        dailyAvgs = self.daily_avgs(var=var, decimals=decimals)
        monthlyMeans = dailyAvgs.groupby(pd.Grouper(freq='M')).mean(numeric_only=True).round(decimals)
        monthlyAvg = monthlyMeans.groupby('Month').mean(numeric_only=True).round(decimals)[var]
        monthlyAvg.index = monthlyAvg.index.astype(int)
        return monthlyAvg

    def monthly_highs(self, var, decimals=1):
        """Monthly highs for variable 'var'."""
        dailyAvgs = self.daily_avgs(var=var)
        return dailyAvgs.groupby(pd.Grouper(freq='M')).max(numeric_only=True)
      
    def monthly_lows(self, var, decimals=1):
        """Monthly lows for variable 'var'."""
        dailyAvgs = self.daily_avgs(var=var)
        return dailyAvgs.groupby(pd.Grouper(freq='M')).min(numeric_only=True)

    def record_high_daily_avg(self, var, decimals=1):
        """Record high daily averages for variable 'var' rounded to 'decimals'."""
        dailyAvgs = self.daily_avgs(var=var, decimals=decimals)
        recordHighDailyAvg = dailyAvgs.groupby('YearDay').max(numeric_only=True).round(decimals)[var]
        recordHighDailyAvg.index = recordHighDailyAvg.index.astype(int)
        recordHighDailyAvgYear = dailyAvgs.groupby('YearDay').idxmax(numeric_only=True)[var].dt.year.astype(int)
        recordHighDailyAvgYear.index = recordHighDailyAvgYear.index.astype(int)
        return (recordHighDailyAvg, recordHighDailyAvgYear)
    
    def record_high_monthly_avg(self, var, decimals=1):
        """Record high monthly averages for variable 'var' rounded to 'decimals'."""
        dailyAvgs = self.daily_avgs(var=var, decimals=decimals)
        monthlyAvgs = dailyAvgs.groupby(pd.Grouper(freq='M')).mean(numeric_only=True).round(decimals)
        recordHighMonthlyAvg = monthlyAvgs.groupby('Month').max(numeric_only=True)[var]
        recordHighMonthlyAvg.index = recordHighMonthlyAvg.index.astype(int)
        recordHighMonthlyAvgYear = monthlyAvgs.groupby('Month').idxmax(numeric_only=True)[var].dt.year.astype(int)
        recordHighMonthlyAvgYear.index = recordHighMonthlyAvgYear.index.astype(int)
        return (recordHighMonthlyAvg, recordHighMonthlyAvgYear)
    
    def record_low_daily_avg(self, var, decimals=1):
        """Record low daily averages for variable 'var' rounded to 'decimals'."""
        dailyAvgs = self.daily_avgs(var=var, decimals=decimals)
        recordLowDailyAvg = dailyAvgs.groupby('YearDay').min(numeric_only=True).round(decimals)[var]
        recordLowDailyAvg.index = recordLowDailyAvg.index.astype(int)
        recordLowDailyAvgYear = dailyAvgs.groupby('YearDay').idxmin(numeric_only=True)[var].dt.year.astype(int)
        recordLowDailyAvgYear.index = recordLowDailyAvg.index.astype(int)
        return (recordLowDailyAvg, recordLowDailyAvgYear)

    def record_low_monthly_avg(self, var, decimals=1):
        """Record low monthly averages for variable 'var' rounded to 'decimals'."""
        dailyAvgs = self.daily_avgs(var=var, decimals=decimals)
        monthlyAvgs = dailyAvgs.groupby(pd.Grouper(freq='M')).mean(numeric_only=True).round(decimals)
        recordLowMonthlyAvg = monthlyAvgs.groupby('Month').min(numeric_only=True)[var]
        recordLowMonthlyAvg.index = recordLowMonthlyAvg.index.astype(int)
        recordLowMonthlyAvgYear = monthlyAvgs.groupby('Month').idxmin(numeric_only=True)[var].dt.year.astype(int)
        recordLowMonthlyAvgYear.index = recordLowMonthlyAvgYear.index.astype(int)
        return (recordLowMonthlyAvg, recordLowMonthlyAvgYear)

    def daily_avg_high(self, var, decimals=1):
        """Average daily highs for variable 'var' rounded to 'decimals'."""        
        dailyHighs = self.daily_highs(var=var, decimals=decimals)
        return dailyHighs.groupby('YearDay').mean(numeric_only=True).round(decimals)[var]
    
    def monthly_avg_high(self, var, decimals=1):
        """Average monthly highs for variable 'var' rounded to 'decimals'."""
        monthlyHighs = self.monthly_highs(var=var, decimals=decimals)
        return monthlyHighs.groupby('Month').mean(numeric_only=True).round(decimals)[var]
    
    def daily_lowest_high(self, var, decimals=1):
        """Lowest daily highs for variable 'var' rounded to 'decimals'."""
        dailyHighs = self.daily_highs(var=var, decimals=decimals)
        lowestHigh = dailyHighs.groupby('YearDay').min(numeric_only=True).round(decimals)[var]
        lowestHighYear = dailyHighs.groupby('YearDay').idxmin(numeric_only=True)[var].dt.year.astype(int)
        return (lowestHigh, lowestHighYear)
    
    def monthly_lowest_high(self, var, decimals=1):
        """Lowest monthly highs for variable 'var' rounded to 'decimals'."""
        monthlyHighs = self.monthly_highs(var=var, decimals=decimals)
        lowestHigh = monthlyHighs.groupby('Month').min(numeric_only=True)[var]
        lowestHighYear = monthlyHighs.groupby('Month').idxmin(numeric_only=True)[var].dt.year.astype(int)
        return (lowestHigh, lowestHighYear)
        
    def daily_record_high(self, var, decimals=1):
        """Record daily highs for variable 'var' rounded to 'decimal'."""
        dailyHighs = self.daily_highs(var=var, decimals=decimals)
        recordHigh = dailyHighs.groupby('YearDay').max(numeric_only=True).round(decimals)[var]
        recordHighYear = dailyHighs.groupby('YearDay').idxmax(numeric_only=True)[var].dt.year.astype(int)
        return (recordHigh, recordHighYear)

    def monthly_record_high(self, var, decimals=1):
        """Record monthly highs for variable 'var' rounded to 'decimals'."""
        monthlyHighs = self.monthly_highs(var=var, decimals=decimals)
        recordHigh = monthlyHighs.groupby('Month').max(numeric_only=True)[var]
        recordHighYear = monthlyHighs.groupby('Month').idxmax(numeric_only=True)[var].dt.year.astype(int)
        return (recordHigh, recordHighYear)

    def daily_avg_low(self, var, decimals=1):
        """Average daily lows for variable 'var' rounded to 'decimals'."""
        dailyLows = self.daily_lows(var=var, decimals=decimals)
        return dailyLows.groupby('YearDay').mean(numeric_only=True).round(decimals)[var]
    
    def monthly_avg_low(self, var, decimals=1):
        """Average monthly lows for variable 'var' rounded to 'decimals'."""
        monthlyLows = self.monthly_lows(var=var, decimals=decimals)
        return monthlyLows.groupby('Month').mean(numeric_only=True).round(decimals)[var]
    
    def daily_highest_low(self, var, decimals=1):
        """Highest daily lows for variable 'var' rounded to 'decimals'."""
        dailyLows = self.daily_lows(var=var, decimals=decimals)
        highestLow =  dailyLows.groupby('YearDay').max(numeric_only=True).round(decimals)[var]
        highestLowYear =  dailyLows.groupby('YearDay').idxmax(numeric_only=True)[var].dt.year.astype(int)
        return (highestLow, highestLowYear)
    
    def monthly_highest_low(self, var, decimals=1):
        """Highest monthly lows for variable 'var' rounded to 'decimals'."""
        monthlyLows = self.monthly_lows(var=var, decimals=decimals)
        highestLow = monthlyLows.groupby('Month').max(numeric_only=True)[var]
        highestLowYear = monthlyLows.groupby('Month').idxmax(numeric_only=True)[var].dt.year.astype(int)
        return (highestLow, highestLowYear)
    
    def daily_record_low(self, var, decimals=1):
        """Record daily lows for variable 'var' rounded to 'decimals'."""
        dailyLows = self.daily_lows(var=var, decimals=decimals)
        recordLow = dailyLows.groupby('YearDay').min(numeric_only=True).round(decimals)[var]
        recordLowYear = dailyLows.groupby('YearDay').idxmin(numeric_only=True)[var].dt.year.astype(int)
        return (recordLow, recordLowYear)
    
    def monthly_record_low(self, var, decimals=1):
        """Record monthly lows for variable 'var' rounded to 'decimals'."""
        monthlyLows = self.monthly_lows(var=var, decimals=decimals)
        recordLow = monthlyLows.groupby('Month').min(numeric_only=True)[var]
        recordLowYear = monthlyLows.groupby('Month').idxmin(numeric_only=True)[var].dt.year.astype(int)
        return (recordLow, recordLowYear)
    
    def number_of_years_byday(self, var):
        """Number of years in the historical data record for variable 'var' by day of year."""
        return self.filtered_data[var].groupby('YearDay').apply(lambda x: len(x['Year'].unique()))

    def number_of_years_bymon(self, var):
        """Number of years in the historical data record for variable 'var' by month."""
        return self.filtered_data[var].groupby('Month').apply(lambda x: len(x['Year'].unique()))

    def generate_yeardays(self):
        return pd.date_range(start='2020-01-01',end='2020-12-31', freq='1D').strftime('%d-%b')
    
    def daily_stats(self):
        """Create daily statistics dictionary for all science variables."""
        # Use a leap year to generate a date list
        YearDay = pd.date_range(
            start='2020-01-01',
            end='2020-12-31',
            freq='1D').strftime('%d-%b')
        # New dictionary to fill
        stats_dict = {k:None for k in self.variables}
        for var in stats_dict.keys():
            if self.verbose:
                print(f'Calculating daily {var.lower()} stats')
            decimals = 2 if var=='Water Level' else 1
            
            # Extract data when multiple outputs
            recordHighDailyAvg, recordHighDailyAvgYr = self.record_high_daily_avg(var=var, decimals=decimals)
            recordLowDailyAvg, recordLowDailyAvgYr = self.record_low_daily_avg(var=var, decimals=decimals)
            lowestHigh, lowestHighYr = self.daily_lowest_high(var=var, decimals=decimals)
            recordHigh, recordHighYr = self.daily_record_high(var=var, decimals=decimals)
            highestLow, highestLowYr = self.daily_highest_low(var=var, decimals=decimals)
            recordLow, recordLowYr = self.daily_record_low(var=var, decimals=decimals)
            
            stats_dict[var] = dict({
                # 'DailyHighs': {d: {'obs': o} \
                #      for d,o in zip(self.daily_highs(var=var, decimals=decimals).index.strftime('%Y-%m-%d').to_list(),
                #                     self.daily_highs(var=var, decimals=decimals)[var].to_list())},
                # 'DailyLows': {d: {'obs': o} \
                #      for d,o in zip(self.daily_lows(var=var, decimals=decimals).index.strftime('%Y-%m-%d').to_list(),
                #                     self.daily_lows(var=var, decimals=decimals)[var].to_list())},
                 'Daily Average': {d: {'obs': o} \
                     for d,o in zip(YearDay,
                                    self.daily_avg(var=var, decimals=decimals).to_list())},
                'Record High Daily Average': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(YearDay,
                                      recordHighDailyAvg.to_list(),
                                      recordHighDailyAvgYr.to_list())},
                'Record Low Daily Average': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(YearDay,
                                      recordLowDailyAvg.to_list(),
                                      recordLowDailyAvgYr.to_list())},
                'Average High': {d: {'obs': o} \
                     for d,o in zip(YearDay,
                                    self.daily_avg_high(var=var, decimals=decimals).to_list())},
                'Lowest High': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(YearDay,
                                      lowestHigh.to_list(),
                                      lowestHighYr.to_list())},
                'Record High': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(YearDay,
                                      recordHigh.to_list(),
                                      recordHighYr.to_list())},
                'Average Low': {d: {'obs': o} \
                     for d,o in zip(YearDay,
                                      self.daily_avg_low(var=var, decimals=decimals).to_list())},
                'Highest Low': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(YearDay,
                                      highestLow.to_list(),
                                      highestLowYr.to_list())},
                'Record Low': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(YearDay,
                                      recordLow.to_list(),
                                      recordLowYr.to_list())},
                'Number of Years': {d: {'obs': o} \
                     for d,o in zip(YearDay,
                                      self.number_of_years_byday(var=var).to_list())}
            })
        
        # Create table version
        self.daily_stats_tables = self.stats_table(stats_dict)
        return stats_dict    

    def monthly_stats(self):
        """Create monthly statistics dictionary for all science variables."""
        # Use a leap year to generate a date list
        months = pd.date_range(
            start='2020-01-01',
            end='2020-12-31',
            freq='1M').strftime('%b')
        # New dictionary to fill
        stats_dict = {k:None for k in self.variables}
        for var in stats_dict.keys():
            if self.verbose:
                print(f'Calculating monthly {var.lower()} stats')
            decimals = 2 if var=='Water Level' else 1
            
            # Extract data when multiple outputs
            recordHighMonthlyAvg, recordHighMonthlyAvgYr = self.record_high_monthly_avg(var=var, decimals=decimals)
            recordLowMonthlyAvg, recordLowMonthlyAvgYr = self.record_low_monthly_avg(var=var, decimals=decimals)
            lowestHigh, lowestHighYr = self.monthly_lowest_high(var=var, decimals=decimals)
            recordHigh, recordHighYr = self.monthly_record_high(var=var, decimals=decimals)
            highestLow, highestLowYr = self.monthly_highest_low(var=var, decimals=decimals)
            recordLow, recordLowYr = self.monthly_record_low(var=var, decimals=decimals)
            
            stats_dict[var] = dict({
                # 'MonthlyHighs': {d: {'obs': o} \
                #      for d,o in zip(self.monthly_highs(var=var, decimals=decimals).index.strftime('%Y-%m-%d').to_list(),
                #                     self.monthly_highs(var=var, decimals=decimals)[var].to_list())},
                # 'MonthlyLows': {d: {'obs': o} \
                #      for d,o in zip(self.daily_lows(var=var, decimals=decimals).index.strftime('%Y-%m-%d').to_list(),
                #                     self.daily_lows(var=var, decimals=decimals)[var].to_list())},
                 'Monthly Average': {d: {'obs': o} \
                     for d,o in zip(months,
                                    self.monthly_avg(var=var, decimals=decimals).to_list())},
                'Record High Monthly Average': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(months,
                                      recordHighMonthlyAvg.to_list(),
                                      recordHighMonthlyAvgYr.to_list())},
                'Record Low Monthly Average': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(months,
                                      recordLowMonthlyAvg.to_list(),
                                      recordLowMonthlyAvgYr.to_list())},
                'Average High': {d: {'obs': o} \
                     for d,o in zip(months,
                                    self.monthly_avg_high(var=var, decimals=decimals).to_list())},
                'Lowest High': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(months,
                                      lowestHigh.to_list(),
                                      lowestHighYr.to_list())},
                'Record High': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(months,
                                      recordHigh.to_list(),
                                      recordHighYr.to_list())},
                'Average Low': {d: {'obs': o} \
                     for d,o in zip(months,
                                      self.monthly_avg_low(var=var, decimals=decimals).to_list())},
                'Highest Low': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(months,
                                      highestLow.to_list(),
                                      highestLowYr.to_list())},
                'Record Low': {d: {'obs': o, 'year':y} \
                     for d,o,y in zip(months,
                                      recordLow.to_list(),
                                      recordLowYr.to_list())},
                'Number of Years': {d: {'obs': o} \
                     for d,o in zip(months,
                                    self.number_of_years_bymon(var=var).to_list())}
            })
        
        # Create table version
        self.monthly_stats_tables = self.stats_table(stats_dict)
        return stats_dict    
    
    def stats_table(self, stats_dict):
        """Convert statistics dictionary to DataFrame for easier viewing"""
        _tables = {k:None for k in self.variables}
        for var in self.variables:
            out = pd.DataFrame.from_dict(stats_dict[var])
            for col in out.columns:
                new = pd.json_normalize(out[col])
                if len(new.columns) == 2:
                    new.columns = [col, col+' Year']
                else:
                    new.columns = [col]
                new.index = out.index
                out.drop(col, axis=1, inplace=True)
                out = pd.concat((out, new), axis=1)
            _tables[var] = out
        return _tables

    def _report(self, record, variable, ondate, values, years):
        """Print new records"""
        oldRecord = values['old_value']
        oldYear = years['old_value']
        newRecord = values['new_value']        
        units = self.units[variable]
        print(f"{record.capitalize()} {variable.lower()} set {ondate}:\n\t"\
              f"{newRecord} {units} (previously {oldRecord} {units} in {oldYear})")        
        
    def _compare(self, d1, d2):
        """Compare dictionaries excluding daily highs, lows, and averages,
        since these will always change with updated data"""
        exclude = [f"root['{v}']['{e}']" \
                   for e in ['Daily Average', 'Monthly Average',
                             'Average High', 'Average Low',
                             'Number of Years'] \
                   for v in self.variables]
        deltas = DeepDiff(d1, d2, exclude_paths=exclude)
        # Print any new records
        if deltas:
            keylist = list(deltas['values_changed'].keys())
            for i in np.arange(0, len(keylist), 2):
                variable, record, ondate, _ = keylist[i][6:].split("']['")
                self._report(record, variable, ondate,
                             values=deltas['values_changed'][keylist[i]],
                             years=deltas['values_changed'][keylist[i+1]])
        
    def get_daily_stats(self, var=None):
        """Return the daily statistics dictionary"""
        try:
            return self.daily_stats_dict[var] if var else self.daily_stats_dict
        except AttributeError:
            raise AttributeError('Instance of Data has no daily stats yet. Run Data.stats() '\
                                 'to calculate stats and try again.')
    
    def get_monthly_stats(self, var=None):
        """Return the monthly statistics dictionary"""
        try:
            return self.monthly_stats_dict[var] if var else self.monthly_stats_dict
        except AttributeError:
            raise AttributeError('Instance of Data has no monthly stats yet. Run Data.stats() '\
                                 'to calculate stats and try again.')

    def get_daily_stats_table(self, var=None):
        """Return the daily statistics table"""
        try:
            return self.daily_stats_tables[var] if var else self.daily_stats_tables
        except AttributeError:
            raise AttributeError('Instance of Data has no daily stats table yet. Run '\
                                 'Data.daily_stats() to calculate stats and try again.')
            
    def get_monthly_stats_table(self, var=None):
        """Return the monthly statistics table"""
        try:
            return self.monthly_stats_tables[var] if var else self.monthly_stats_tables
        except AttributeError:
            raise AttributeError('Instance of Data has no monthly stats table yet. Run '\
                                 'Data.monthly_stats() to calculate stats and try again.')

    def _skip_keys(self, d, keys):
        return {x: d[x] for x in d if x not in keys}

    def set_station(self, station):
        self.name = station
        
    def get_station(self):
        return self.name
    
    def get_stationid(self):
        return self.id
    
    def get_variables(self):
        return self.variables
    
    def __str__(self):
        return("Oceanic and atmospheric observations for station "\
               f"'{self.name}' (station ID {self.id}):\n"\
               f"{self.station.data_inventory}")
    
    def __repr__(self):
        return(f"{type(self).__name__}("\
               f"stationname='{self.name}', stationid='{self.id}', outdir='{self.outdir}', "\
               f"timezone='{self.tz}', units='{self.units}', datum='{self.datum}', "\
               f"hr_threshold='{self.hr_threshold}', day_threshold='{self.day_threshold}')")

***

In [1]:
from collections import OrderedDict
from noaa_coops import Station
from deepdiff import DeepDiff
import datetime as dt
import xarray as xr
import pandas as pd
import numpy as np
import calendar
import os
import json
import plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [2]:
def _camel(text):
    """Convert to camel case"""
    s = text.replace(',','').replace("-", " ").replace("_", " ")
    s = s.split()
    if len(text) == 0:
        return text
    return s[0].lower() + ''.join(i.capitalize() for i in s[1:])

def _format_date(datestr):
    dtdt = pd.to_datetime(datestr)
    return dt.datetime.strftime(dtdt, '%Y%m%d')

In [3]:
stationname = 'Virginia Key, FL'
dirname = _camel(stationname)
id = '8723214'
units = 'english'
unit_system = units.lower()
timezone = 'lst'
tz = timezone.lower()
datum = 'MHHW'
datum = datum.upper()
hr_threshold = 3
day_threshold = 2
verbose = True
variables = []
today = _format_date(pd.to_datetime('today'))
outdir = os.path.join(os.getcwd(), dirname)

In [4]:
    def stats_table(stats_dict):
        """Convert statistics dictionary to DataFrame for easier viewing"""
        _tables = {k:None for k in variables}
        for var in variables:
            out = pd.DataFrame.from_dict(stats_dict[var])
            for col in out.columns:
                new = pd.json_normalize(out[col])
                if len(new.columns) == 2:
                    new.columns = [col, col+' Year']
                else:
                    new.columns = [col]
                new.index = out.index
                out.drop(col, axis=1, inplace=True)
                out = pd.concat((out, new), axis=1)
            _tables[var] = out
        return _tables


In [5]:
    def _DOY(df):
        """Calculate year day out of 366"""
        import calendar
        # Day of year as integer
        df['YearDay'] = df.index.day_of_year.astype(int)
        # Years that are NOT leap years
        leapInd = [not calendar.isleap(i) for i in df.index.year]
        mask = (leapInd) & (df['Month'] > 2)
        # Advance by one day everything after February 28 
        df.loc[mask, 'YearDay'] += 1
        # Day of year for plotting
        # df['DOY'] = pd.to_datetime(df['YearDay'].astype(str), format="%j")
        df['DOY'] = pd.to_datetime(df.index.day_of_year.astype(str), format="%j")
        return df

    def _set_dayflag(data, threshold_hrs=3):
        """Set flag indicating whether each day contains at least
        (24 - 'threshold_hours') hours of data: True if yes, False if not.
        """
        def _flag(group, threshold=threshold_hrs):
            # gap = sum(group.index.to_series().diff() > pd.Timedelta(hours=1))
            gap = group.resample('1h').mean(numeric_only=True).isna().sum()
            return gap <= threshold

        newColName = data.name + '_DayFlag'
        data = pd.DataFrame(data[:])
        data[newColName] = data.groupby(pd.Grouper(freq='D')).agg(lambda x: _flag(group=x, threshold=threshold_hrs))
        data[newColName] = data.resample('D')[newColName].ffill()
        data = data.astype({newColName: 'bool'})
        return data
    
    def _set_monthflag(data, threshold_days=2):
        """Set flag indicating whether each month contains at least
        (n -'threshold_days') days of data, where n is the number of
        days in the given month: True if yes, False if not.
        """
        def _flag(group, threshold=threshold_days):
            try:
                # gap = sum(group.index.to_series().diff() > pd.Timedelta(days=1))
                # flag = gap <= threshold
                daysWithData = group.resample('1D').mean(numeric_only=True).dropna().size
                daysNoData =  pd.Period(group.index[0].strftime(format='%Y-%m-%d')).days_in_month - daysWithData
                flag = daysNoData <= threshold
                return pd.DataFrame(pd.Series(flag, index=group.index))
            except IndexError:
                return None

        origColName = data.name
        newColName = data.name + '_MonthFlag'
        data = pd.DataFrame(data[:])
        data['Year'] = data.index.year
        data['Month'] = data.index.month
        data['Day'] = data.index.day
        data[newColName] = data.groupby(['Year', 'Month'], group_keys=False)\
                               .apply(lambda x: _flag(x, threshold=threshold_days))
        return data

    
    def _filter_data(series, hr_threshold=3, day_threshold=2):
        """Removes days with more than 'hr_threshold' of missing data
        and months with more than 'day_threshold' days of missing data.
        """
        # Variable to process
        var = series.name

        # Remove days with >hr_threshold hours of missing data
        df_dayFlagged = _set_dayflag(series[:], threshold_hrs=hr_threshold)
        dfss = df_dayFlagged.loc[df_dayFlagged[f'{var}_DayFlag'],:]

        # Remove months with >day_threshold days of missing data
        df_monFlagged = _set_monthflag(dfss[var], threshold_days=day_threshold)
        dfss = df_monFlagged.loc[df_monFlagged[f'{var}_MonthFlag'],:].copy()
        
        # Add Year Day
        dfss = _DOY(dfss)

        # Drop month flag
        dfss.drop(f'{var}_MonthFlag', axis=1, inplace=True)
        return dfss.sort_index()


In [6]:
if verbose:
    print('Loading metadata from file')
with open(os.path.join(outdir, 'metadata.json')) as m:
    meta = json.load(m)
# _load_from_json(meta)

# Load the historical data from file
if verbose:
    print('Loading historical data from file')
data = pd.read_csv(os.path.join(outdir, 'observational_data_record.csv.gz'),
                        index_col=f'time_{tz}', parse_dates=True,
                        compression='infer')

variables = data.columns

# Load daily statistics from file
if verbose:
    print('Loading daily statistics from file')
with open(os.path.join(outdir, 'statistics-daily.json')) as s:
    daily_stats_dict = json.load(s)
daily_stats_tables = stats_table(daily_stats_dict)

# Load monthly statistics from file
if verbose:
    print('Loading monthly statistics from file')
with open(os.path.join(outdir, 'statistics-monthly.json')) as s:
    monthly_stats_dict = json.load(s)
monthly_stats_tables = stats_table(monthly_stats_dict)

# Clean and format
if verbose:
    print('Filtering data')
filtered_data = {var: _filter_data(data[var],
                                             hr_threshold=hr_threshold,
                                             day_threshold=day_threshold) \
                      for var in variables}     

if verbose:
    print('Done!')

Loading metadata from file
Loading historical data from file
Loading daily statistics from file
Loading monthly statistics from file
Filtering data


/tmp/ipykernel_107/279509391.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: _flag(x, threshold=threshold_days))
/tmp/ipykernel_107/279509391.py:8: FutureWarning: Logical ops (and, or, xor) between Pandas objects and dtype-less sequences (e.g. list, tuple) are deprecated and will raise in a future version. Wrap the object in a Series, Index, or np.array before operating instead.
  mask = (leapInd) & (df['Month'] > 2)
/tmp/ipykernel_107/279509391.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=

Done!


In [7]:
    def _set_dayflag(data, threshold_hrs=3):
        """Set flag indicating whether each day contains at least
        (24 - 'threshold_hours') hours of data: True if yes, False if not.
        """
        def _flag(group, threshold=threshold_hrs):
            # gap = sum(group.index.to_series().diff() > pd.Timedelta(hours=1))
            gap = group.resample('1h').mean(numeric_only=True).isna().sum()
            return gap <= threshold

        newColName = data.name + '_DayFlag'
        data = pd.DataFrame(data[:])
        data[newColName] = data.groupby(pd.Grouper(freq='D')).agg(lambda x: _flag(group=x, threshold=threshold_hrs))
        data[newColName] = data.resample('D')[newColName].ffill()
        data = data.astype({newColName: 'bool'})
        return data

In [10]:
import warnings
warnings.filterwarnings('always')

`data` is the historical data saved to file. Copy this to work on it.

In [8]:
data2 = data.copy()

***

Revised day filtering to test

In [11]:
def count_missing_hours(group, threshold=3):
    # Returns True the number of hours in a day with missing data is less than or equal to the threshold and False otherwise.
    missing_hours = group.resample('1h').mean().isna().sum()
    return missing_hours <= threshold

In [12]:
test2 = data2['Air Temperature'].groupby(pd.Grouper(freq='1D')).filter(lambda x: count_missing_hours(group=x))

In [13]:
test2

time_lst
1994-01-29 00:00:00    75.0
1994-01-29 00:06:00     NaN
1994-01-29 00:12:00     NaN
1994-01-29 00:18:00     NaN
1994-01-29 00:24:00     NaN
                       ... 
2023-10-18 20:36:00    77.9
2023-10-18 20:42:00    78.1
2023-10-18 20:48:00    77.9
2023-10-18 20:54:00    77.9
2023-10-18 21:00:00    77.9
Name: Air Temperature, Length: 2235602, dtype: float64

This seems to work...

***

Revised month filtering

In [14]:
def count_missing_days(group, threshold=2):
    # Returns True if the number of days in a month with missing data is less than or equal to the theshold and False otherwise.
    # Need to perform two tests: missing data (NaN) and compare to the number of days in the given month
    try:
        days_in_month = pd.Period(group.index[0].strftime(format='%Y-%m-%d')).days_in_month
        missing_days = group.resample('1D').mean().isna().sum()
        missing_days_flag = missing_days <= threshold
        days_in_month_flag = days_in_month - group.resample('1D').mean().size <= threshold
        return min(missing_days_flag, days_in_month_flag)
    except IndexError:
        pass

In [18]:
test3 = test2.groupby(pd.Grouper(freq='1ME')).filter(lambda x: count_missing_days(group=x))

In [20]:
test3

time_lst
1994-04-01 00:00:00    73.2
1994-04-01 00:06:00     NaN
1994-04-01 00:12:00     NaN
1994-04-01 00:18:00     NaN
1994-04-01 00:24:00     NaN
                       ... 
2023-09-30 23:30:00    82.0
2023-09-30 23:36:00    81.9
2023-09-30 23:42:00    81.9
2023-09-30 23:48:00    82.2
2023-09-30 23:54:00    82.4
Name: Air Temperature, Length: 1965278, dtype: float64

This seems to work.

***

Wrap these new filtering routines into a single function

In [36]:
%%time

for var in variables:
    # Filter out days missing more than 3 hours
    test4 = data2[var].groupby(pd.Grouper(freq='1D')).filter(lambda x: count_missing_hours(group=x))

    # Filter out months missing more than 2 days
    test4 = test4.groupby(pd.Grouper(freq='1ME')).filter(lambda x: count_missing_days(group=x))

CPU times: user 11.3 s, sys: 28.2 ms, total: 11.3 s
Wall time: 11.3 s


In [37]:
test4.head()

time_lst
1994-04-01 00:00:00    75.7
1994-04-01 00:06:00     NaN
1994-04-01 00:12:00     NaN
1994-04-01 00:18:00     NaN
1994-04-01 00:24:00     NaN
Name: Water Temperature, dtype: float64

In [94]:
def filter_data(data, hr_threshold=3, day_threshold=2):
    # Filter out days missing more than <hr_threshold> hours
    filtered = data.groupby(pd.Grouper(freq='1D')).filter(lambda x: count_missing_hours(group=x, threshold=hr_threshold))
    # Filter out months missing more than <day_threshold> days
    filtered = filtered.groupby(pd.Grouper(freq='1ME')).filter(lambda x: count_missing_days(group=x, threshold=day_threshold))
    return filtered

In [23]:
%%time
for var in variables:
    test5 = filter_data(data2[var])

CPU times: user 11.3 s, sys: 71.6 ms, total: 11.3 s
Wall time: 11.3 s


In [24]:
test5.head()

time_lst
1994-04-01 00:00:00    75.7
1994-04-01 00:06:00     NaN
1994-04-01 00:12:00     NaN
1994-04-01 00:18:00     NaN
1994-04-01 00:24:00     NaN
Name: Water Temperature, dtype: float64

In [38]:
test4.equals(test5)

True

Compare to the original method:

In [288]:
%%time

filtered_data = {var: _filter_data(data[var],
                                             hr_threshold=hr_threshold,
                                             day_threshold=day_threshold) \
                      for var in variables}

17.3 s ± 174 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Keeping the new one:

In [39]:
# Revised filtered data

filtered_data = {var: filter_data(data[var],
                                  hr_threshold=hr_threshold,
                                  day_threshold=day_threshold) \
                      for var in variables}

In [265]:
group = test2.groupby(pd.Grouper(freq='1M')).get_group(pd.to_datetime('1994-04-30'))

In [167]:
    def _DOY(df):
        """Calculate year day out of 366"""
        import calendar
        # Day of year as integer
        df['YearDay'] = df.index.day_of_year.astype(int)
        # Years that are NOT leap years
        leapInd = [not calendar.isleap(i) for i in df.index.year]
        mask = (leapInd) & (df.index.month > 2)
        # Advance by one day everything after February 28 
        df.loc[mask, 'YearDay'] += 1
        # Day of year for plotting
        # df['DOY'] = df.index.strftime('%m-%d')
        return df

In [87]:
    def daily_highs(df, decimals=1):
        """Daily highs for variable 'var'."""
        return df.groupby(pd.Grouper(freq='D')).max(numeric_only=True)
    
    def daily_lows(df, decimals=1):
        """Daily lows for variable 'var'."""
        return df.groupby(pd.Grouper(freq='D')).min(numeric_only=True)

    def daily_avgs(df, decimals=1, true_average=False):
        """Daily averages by calendar day for variable 'var' rounded to 'decimals'.
        If 'true_average' is True, all measurements from each 24-hour day will be
        used to calculate the average. Otherwise, only the maximum and minimum
        observations are used. Defaults to False.
        """
        if true_average:
            return df.groupby(pd.Grouper(freq='D')).mean(numeric_only=True).round(decimals)
        else:
            dailyHighs = daily_highs(df=df, decimals=decimals)
            dailyLows = daily_lows(df=df, decimals=decimals)
            return ((dailyHighs + dailyLows)/2).round(decimals)

    def daily_avg(df, decimals=1):
        """Daily averages for variable 'var' rounded to 'decimals'."""
        dailyAvgs = daily_avgs(df=df, decimals=decimals)
        dailyAvg = dailyAvgs.groupby('YearDay').mean(numeric_only=True).round(decimals)
        dailyAvg.index = dailyAvg.index.astype(int)
        results = xr.DataArray(dailyAvg, dims=['yearday', 'variable'])
        results.name = 'Daily Average'
        return results

In [169]:
airfiltered = filtered_data['Air Temperature']
wtrfiltered = filtered_data['Water Temperature']
filtered_data = pd.concat((airfiltered, wtrfiltered), axis=1)
filtered_data = _DOY(filtered_data)
filtered_data.head()

,Air Temperature,Water Temperature,YearDay
time_lst,,,
1994-04-01 00:00:00,73.2,75.7,92
1994-04-01 00:06:00,NaN,NaN,92
1994-04-01 00:12:00,NaN,NaN,92
1994-04-01 00:18:00,NaN,NaN,92
1994-04-01 00:24:00,NaN,NaN,92


In [93]:
data2.head()

,Air Temperature,Water Temperature
time_lst,,
1994-01-28 00:00:00,NaN,NaN
1994-01-28 00:06:00,NaN,NaN
1994-01-28 00:12:00,NaN,NaN
1994-01-28 00:18:00,NaN,NaN
1994-01-28 00:24:00,NaN,NaN


In [110]:
filter_data(data2[var])

time_lst
1994-04-01 00:00:00    75.7
1994-04-01 00:06:00     NaN
1994-04-01 00:12:00     NaN
1994-04-01 00:18:00     NaN
1994-04-01 00:24:00     NaN
                       ... 
2023-09-30 23:30:00    86.5
2023-09-30 23:36:00    86.5
2023-09-30 23:42:00    86.5
2023-09-30 23:48:00    86.7
2023-09-30 23:54:00    86.5
Name: Water Temperature, Length: 2028955, dtype: float64

In [98]:
filtered_data = pd.concat([filter_data(data2[var],
                                       hr_threshold=hr_threshold,
                                       day_threshold=day_threshold)
                                       for var in variables], axis=1)
filtered_data = _DOY(filtered_data)

In [113]:
filtered_data2 = filtered_data.to_xarray()

In [148]:
    def daily_highs(df, decimals=1):
        """Daily highs for variable 'var'."""
        return df.groupby(pd.Grouper(freq='1D')).max(numeric_only=True)
    
    def daily_lows(df, decimals=1):
        """Daily lows for variable 'var'."""
        return df.groupby(pd.Grouper(freq='1D')).min(numeric_only=True)

In [155]:
filtered_data.head()

,Air Temperature,Water Temperature,YearDay,DOY
time_lst,,,,
1994-04-01 00:00:00,73.2,75.7,92,1900-04-01
1994-04-01 00:06:00,NaN,NaN,92,1900-04-01
1994-04-01 00:12:00,NaN,NaN,92,1900-04-01
1994-04-01 00:18:00,NaN,NaN,92,1900-04-01
1994-04-01 00:24:00,NaN,NaN,92,1900-04-01


In [101]:
    def record_high_daily_avg(df, decimals=1):
        """Record high daily averages for variable 'var' rounded to 'decimals'."""
        # Calculate the records
        dailyAvgs = daily_avgs(df=df, decimals=decimals)
        recordHighDailyAvg = dailyAvgs.groupby('YearDay').max(numeric_only=True).round(decimals)
        recordHighDailyAvg.index = recordHighDailyAvg.index.astype(int)
        # Record years
        recordHighDailyAvgYear = dailyAvgs.groupby('YearDay').apply(lambda x: x.idxmax(numeric_only=True).dt.year, include_groups=False)
        recordHighDailyAvgYear.index = recordHighDailyAvgYear.index.astype(int)
        recordHighDailyAvgYear.columns = [i+' Year' for i in recordHighDailyAvgYear]
        # Create an xarray DataArray
        results = pd.concat((recordHighDailyAvg, recordHighDailyAvgYear), axis=1)
        results = xr.DataArray(results, dims=['yearday', 'variable'])
        results.name = 'Record High Daily Average'
        return results

In [102]:
daily_avg(filtered_data).head()

<xarray.DataArray 'Daily Average' (yearday: 5, variable: 2)> Size: 80B
array([[71.7, 72.5],
       [72. , 73. ],
       [70.2, 73.1],
       [69.2, 72.8],
       [68.2, 72.5]])
Coordinates:
  * yearday   (yearday) int64 40B 1 2 3 4 5
  * variable  (variable) object 16B 'Air Temperature' 'Water Temperature'

In [103]:
record_high_daily_avg(filtered_data)

<xarray.DataArray 'Record High Daily Average' (yearday: 366, variable: 4)> Size: 12kB
array([[  78. ,   80.4, 2022. , 2022. ],
       [  77.8,   80.2, 2022. , 2022. ],
       [  78. ,   80.7, 2015. , 2017. ],
       ...,
       [  79.4,   82.6, 2015. , 2016. ],
       [  79.5,   81.9, 2015. , 2016. ],
       [  79.2,   80.5, 2015. , 2021. ]])
Coordinates:
  * yearday   (yearday) int64 3kB 1 2 3 4 5 6 7 ... 360 361 362 363 364 365 366
  * variable  (variable) object 32B 'Air Temperature' ... 'Water Temperature ...

So, we can assemble a dataset of daily records by calling each daily record function and storing station metadata.
Same for monthly records.

Next steps:

1. Rewrite all functions to produce xarrays
2. Compile into dataset to test
3. Continue with rest of routine

In [370]:
def daily_highs(df):
    """Daily highs"""
    return df.groupby(pd.Grouper(freq='1D')).max(numeric_only=True)

def daily_lows(df):
    """Daily lows"""
    return df.groupby(pd.Grouper(freq='1D')).min(numeric_only=True)

def daily_avgs(df, decimals=1, true_average=False):
    """Daily averages by calendar day rounded to 'decimals'. If
    'true_average' is True, all measurements from each 24-hour day will be
    used to calculate the average. Otherwise, only the maximum and minimum
    observations are used. Defaults to False (meteorological standard).
    """
    if true_average:
        results = df.groupby(pd.Grouper(freq='1D')).mean(numeric_only=True)
    else:
        dailyHighs = daily_highs(df)
        dailyLows = daily_lows(df)
        results = (dailyHighs + dailyLows) / 2
    return results.round(decimals)

def daily_avg(df, decimals=1, true_average=False):
    """Daily averages rounded to 'decimals'. If 'true_average' is True, all
    measurements from each 24-hour day will be used to calculate the daily
    average. Otherwise, only the maximum and minimum observations are used.
    Defaults to False (meteorological standard).
    """
    dailyAvgs = daily_avgs(df, decimals=decimals, true_average=true_average)
    dailyAvg = dailyAvgs.groupby('YearDay').mean(numeric_only=True).round(decimals)
    dailyAvg.index = dailyAvg.index.astype(int)
    results = xr.DataArray(dailyAvg, dims=['yearday', 'variable'])
    results.name = 'Daily Average'
    return results

def monthly_highs(df, decimals=1, true_average=False):
    """Monthly highs rounded to 'decimals'. If 'true_average' is True, all
    measurements from each 24-hour day will be used to calculate the daily
    average. Otherwise, only the maximum and minimum observations are used.
    Defaults to False (meteorological standard).
    """
    dailyAvgs = daily_avgs(df, decimals=decimals, true_average=False)
    monthHighs = dailyAvgs.groupby(pd.Grouper(freq='ME')).max(numeric_only=True)
    return monthHighs
  
def monthly_lows(df, decimals=1, true_average=False):
    """Monthly lows rounded to 'decimals'. If 'true_average' is True, all
    measurements from each 24-hour day will be used to calculate the daily
    average. Otherwise, only the maximum and minimum observations are used.
    Defaults to False (meteorological standard).
    """
    dailyAvgs = daily_avgs(df, decimals=decimals, true_average=true_average)
    monthLows = dailyAvgs.groupby(pd.Grouper(freq='ME')).min(numeric_only=True)
    return monthLows
    
def monthly_avg(df, decimals=1, true_average=False):
    """Monthly averages for variable 'var' rounded to 'decimals'. If
    'true_average' is True, all measurements from each 24-hour day will be
    used to calculate the daily average. Otherwise, only the maximum and
    minimum observations are used. Defaults to False (meteorological
    standard).
    """
    dailyAvgs = daily_avgs(df, decimals=decimals, true_average=true_average)
    monthlyMeans = dailyAvgs.groupby(pd.Grouper(freq='ME')).mean(numeric_only=True).round(decimals)
    monthlyMeans.drop('YearDay', axis=1, inplace=True)
    monthlyAvg = monthlyMeans.groupby(monthlyMeans.index.month).mean(numeric_only=True).round(decimals)
    monthlyAvg.index = monthlyAvg.index.astype(int)
    results = xr.DataArray(monthlyAvg, dims=['month', 'variable'])
    results.name = 'Monthly Average'
    return results

def record_high_daily_avg(df, decimals=1, true_average=False):
    """Record high daily averages rounded to 'decimals'. If 'true_average'
    is True, all measurements from each 24-hour day will be used to
    calculate the daily average. Otherwise, only the maximum and minimum
    observations are used. Defaults to False (meteorological standard).
    """
    # Calculate the records
    dailyAvgs = daily_avgs(df=df, decimals=decimals, true_average=true_average)
    recordHighDailyAvg = dailyAvgs.groupby('YearDay').max(numeric_only=True).round(decimals)
    recordHighDailyAvg.index = recordHighDailyAvg.index.astype(int)
    # Record years
    recordHighDailyAvgYear = dailyAvgs.groupby('YearDay').apply(lambda x: x.idxmax(numeric_only=True).dt.year, include_groups=False)
    recordHighDailyAvgYear.index = recordHighDailyAvgYear.index.astype(int)
    recordHighDailyAvgYear.columns = [i+' Year' for i in recordHighDailyAvgYear.columns]
    # Create xarray
    results = pd.concat((recordHighDailyAvg, recordHighDailyAvgYear), axis=1)
    results = xr.DataArray(results, dims=['yearday', 'variable'])
    results.name = 'Record High Daily Average'
    return results
    
def record_high_monthly_avg(df, decimals=1, true_average=False):
    """Record high monthly averages rounded to 'decimals'. If
    'true_average' is True, all measurements from each 24-hour day will be
    used to calculate the daily average. Otherwise, only the maximum and
    minimum observations are used. Defaults to False (meteorological
    standard).
    """
    # Calculate the records
    dailyAvgs = daily_avgs(df, decimals=decimals, true_average=true_average)
    monthlyAvgs = dailyAvgs.groupby(pd.Grouper(freq='ME')).mean(numeric_only=True).round(decimals)
    monthlyAvgs.drop('YearDay', axis=1, inplace=True)
    recordHighMonthlyAvg = monthlyAvgs.groupby(monthlyAvgs.index.month).max(numeric_only=True)
    recordHighMonthlyAvg.index = recordHighMonthlyAvg.index.astype(int)
    # Record years
    recordHighMonthlyAvgYear = monthlyAvgs.groupby(monthlyAvgs.index.month).apply(lambda x: x.idxmax(numeric_only=True).dt.year, include_groups=False)
    recordHighMonthlyAvgYear.index = recordHighMonthlyAvgYear.index.astype(int)
    recordHighMonthlyAvgYear.columns = [i+' Year' for i in recordHighMonthlyAvgYear.columns]
    # Create xarray
    results = pd.concat((recordHighMonthlyAvg, recordHighMonthlyAvgYear), axis=1)
    results = xr.DataArray(results, dims=['month', 'variable'])
    results.name = 'Record High Monthly Average'
    return results

def record_low_daily_avg(df, decimals=1, true_average=False):
    """Record low daily averages rounded to 'decimals'.  If 'true_average'
    True, all measurements from each 24-hour day will be used to calculate
    the average. Otherwise, only the maximum and minimum observations are
    used. Defaults to False (meteorological standard)."""
    # Calculate the records
    dailyAvgs = daily_avgs(df=df, decimals=decimals, true_average=true_average)
    recordLowDailyAvg = dailyAvgs.groupby('YearDay').min(numeric_only=True).round(decimals)
    recordLowDailyAvg.index = recordLowDailyAvg.index.astype(int)
    # Record years
    recordLowDailyAvgYear = dailyAvgs.groupby('YearDay').apply(lambda x: x.idxmin(numeric_only=True).dt.year, include_groups=False)
    recordLowDailyAvgYear.index = recordLowDailyAvgYear.index.astype(int)
    recordLowDailyAvgYear.columns = [i+' Year' for i in recordLowDailyAvgYear.columns]
    # Create xarray
    results = pd.concat((recordLowDailyAvg, recordLowDailyAvgYear), axis=1)
    results = xr.DataArray(results, dims=['yearday', 'variable'])
    results.name = 'Record Low Daily Average'
    return results

def record_low_monthly_avg(df, decimals=1, true_average=False):
    """Record low monthly averages rounded to 'decimals'. If 'true_average'
    is True, all measurements from each 24-hour day will be used to
    calculate the daily average. Otherwise, only the maximum and minimum
    observations are used. Defaults to False (meteorological standard).
    """
    # Calculate the records
    dailyAvgs = daily_avgs(df, decimals=decimals, true_average=true_average)
    monthlyAvgs = dailyAvgs.groupby(pd.Grouper(freq='ME')).mean(numeric_only=True).round(decimals)
    monthlyAvgs.drop('YearDay', axis=1, inplace=True)
    recordLowMonthlyAvg = monthlyAvgs.groupby(monthlyAvgs.index.month).min(numeric_only=True)
    recordLowMonthlyAvg.index = recordLowMonthlyAvg.index.astype(int)
    # Record years
    recordLowMonthlyAvgYear = monthlyAvgs.groupby(monthlyAvgs.index.month).apply(lambda x: x.idxmin(numeric_only=True).dt.year, include_groups=False)
    recordLowMonthlyAvgYear.index = recordLowMonthlyAvgYear.index.astype(int)
    recordLowMonthlyAvgYear.columns = [i+' Year' for i in recordLowMonthlyAvgYear.columns]
    # Create xarray
    results = pd.concat((recordLowMonthlyAvg, recordLowMonthlyAvgYear), axis=1)
    results = xr.DataArray(results, dims=['month', 'variable'])
    results.name = 'Record Low Monthly Average'
    return results

def avg_daily_high(df, decimals=1):
    """Average daily highs rounded to 'decimals'."""        
    dailyHighs = daily_highs(df)
    results = dailyHighs.groupby('YearDay').mean(numeric_only=True).round(decimals)
    results = xr.DataArray(results, dims=['yearday', 'variable'])
    results.name = 'Average Daily High'
    return results

def avg_monthly_high(df, decimals=1, true_average=False):
    """Average monthly highs rounded to 'decimals'. If 'true_average' is
    True, all measurements from each 24-hour day will be used to calculate
    the daily average. Otherwise, only the maximum and minimum observations
    are used. Defaults to False (meteorological standard).
    """
    monthlyHighs = monthly_highs(df, decimals=decimals, true_average=true_average)
    monthlyHighs.drop('YearDay', axis=1, inplace=True)
    avgMonthlyHighs = monthlyHighs.groupby(monthlyHighs.index.month).mean(numeric_only=True).round(decimals)
    results = xr.DataArray(avgMonthlyHighs, dims=['month', 'variable'])
    results.name = 'Average Monthly High'
    return results

def lowest_daily_high(df, decimals=1):
    """Lowest daily highs rounded to 'decimals'."""
    # Calculate the record
    dailyHighs = daily_highs(df)
    lowestHigh = dailyHighs.groupby('YearDay').min(numeric_only=True).round(decimals)
    lowestHigh.index = lowestHigh.index.astype(int)
    # Record years
    lowestHighYear = dailyHighs.groupby('YearDay').apply(lambda x: x.idxmin(numeric_only=True).dt.year, include_groups=False)
    lowestHighYear.index = lowestHighYear.index.astype(int)
    lowestHighYear.columns = [i+' Year' for i in lowestHighYear.columns]
    # Create xarray
    results = pd.concat((lowestHigh, lowestHighYear), axis=1)
    results = xr.DataArray(results, dims=['yearday', 'variable'])
    results.name = 'Lowest Daily High'
    return results
    
def lowest_monthly_high(df, decimals=1, true_average=False):
    """Lowest monthly highs rounded to 'decimals'. If 'true_average' is
    True, all measurements from each 24-hour day will be used to calculate
    the daily average. Otherwise, only the maximum and minimum observations
    are used. Defaults to False (meteorological standard).
    """
    # Calculate the record
    monthlyHighs = monthly_highs(df, decimals=decimals, true_average=true_average)
    monthlyHighs.drop('YearDay', axis=1, inplace=True)
    lowMonthlyHigh = monthlyHighs.groupby(monthlyHighs.index.month).min(numeric_only=True).round(decimals)
    lowMonthlyHigh.index = lowMonthlyHigh.index.astype(int)
    # Record years
    lowMonthlyHighYear = monthlyHighs.groupby(monthlyHighs.index.month).apply(lambda x: x.idxmin(numeric_only=True).dt.year, include_groups=False)
    lowMonthlyHighYear.index = lowMonthlyHighYear.index.astype(int)
    lowMonthlyHighYear.columns = [i+' Year' for i in lowMonthlyHighYear.columns]
    # Create xarray
    results = pd.concat((lowMonthlyHigh, lowMonthlyHighYear), axis=1)
    results = xr.DataArray(results, dims=['month', 'variable'])
    results.name = 'Lowest Monthly High'
    return results

def record_daily_high(df, decimals=1):
    """Record daily highs rounded to 'decimal'."""
    # Calculate the record
    dailyHighs = daily_highs(df)
    recordHigh = dailyHighs.groupby('YearDay').max(numeric_only=True).round(decimals)
    recordHigh.index = recordHigh.index.astype(int)
    # Record years
    recordHighYear = dailyHighs.groupby('YearDay').apply(lambda x: x.idxmax(numeric_only=True).dt.year, include_groups=False)
    recordHighYear.index = recordHighYear.index.astype(int)
    recordHighYear.columns = [i+' Year' for i in recordHighYear.columns]
    # Create xarray
    results = pd.concat((recordHigh, recordHighYear), axis=1)
    results = xr.DataArray(results, dims=['yearday', 'variable'])
    results.name = 'Record Daily High'
    return results

def record_monthly_high(df, decimals=1, true_average=False):
    """Record monthly highs rounded to 'decimals'. If 'true_average' is
    True, all measurements from each 24-hour day will be used to calculate
    the daily average. Otherwise, only the maximum and minimum observations
    are used. Defaults to False (meteorological standard).
    """
    # Calculate the record
    monthlyHighs = monthly_highs(df, decimals=decimals, true_average=true_average)
    monthlyHighs.drop('YearDay', axis=1, inplace=True)
    recordMonthlyHigh = monthlyHighs.groupby(monthlyHighs.index.month).max(numeric_only=True).round(decimals)
    recordMonthlyHigh.index = recordMonthlyHigh.index.astype(int)
    # Record years
    recordMonthlyHighYear = monthlyHighs.groupby(monthlyHighs.index.month).apply(lambda x: x.idxmax(numeric_only=True).dt.year, include_groups=False)
    recordMonthlyHighYear.index = recordMonthlyHighYear.index.astype(int)
    recordMonthlyHighYear.columns = [i+' Year' for i in recordMonthlyHighYear.columns]
    # Create xarray
    results = pd.concat((recordMonthlyHigh, recordMonthlyHighYear), axis=1)
    results = xr.DataArray(results, dims=['month', 'variable'])
    results.name = 'Record Monthly High'
    return results

def avg_daily_low(df, decimals=1):
    """Average daily lows rounded to 'decimals'."""        
    dailyLows = daily_lows(df)
    results = dailyLows.groupby('YearDay').mean(numeric_only=True).round(decimals)
    results = xr.DataArray(results, dims=['yearday', 'variable'])
    results.name = 'Average Daily Low'
    return results

def avg_monthly_low(df, decimals=1, true_average=False):
    """Average monthly lows rounded to 'decimals'. If 'true_average' is
    True, all measurements from each 24-hour day will be used to calculate
    the daily average. Otherwise, only the maximum and minimum observations
    are used. Defaults to False (meteorological standard).
    """
    monthlyLows = monthly_lows(df, decimals=decimals, true_average=true_average)
    monthlyLows.drop('YearDay', axis=1, inplace=True)
    avgMonthlyLows = monthlyLows.groupby(monthlyLows.index.month).mean(numeric_only=True).round(decimals)
    results = xr.DataArray(avgMonthlyLows, dims=['month', 'variable'])
    results.name = 'Average Monthly Low'
    return results

def highest_daily_low(df, decimals=1):
    """Highest daily lows rounded to 'decimals'."""
    # Calculate the record
    dailyLows = daily_lows(df)
    highestLow = dailyLows.groupby('YearDay').max(numeric_only=True).round(decimals)
    highestLow.index = highestLow.index.astype(int)
    # Record years
    highestLowYear = dailyLows.groupby('YearDay').apply(lambda x: x.idxmax(numeric_only=True).dt.year, include_groups=False)
    highestLowYear.index = highestLowYear.index.astype(int)
    highestLowYear.columns = [i+' Year' for i in highestLowYear.columns]
    # Create xarray
    results = pd.concat((highestLow, highestLowYear), axis=1)
    results = xr.DataArray(results, dims=['yearday', 'variable'])
    results.name = 'Highest Daily Low'
    return results
    
def highest_monthly_low(df, decimals=1, true_average=False):
    """Highest monthly lows rounded to 'decimals'. If 'true_average' is
    True, all measurements from each 24-hour day will be used to calculate
    the daily average. Otherwise, only the maximum and minimum observations
    are used. Defaults to False (meteorological standard).
    """
    # Calculate the record
    monthlyLows = monthly_lows(df, decimals=decimals, true_average=true_average)
    monthlyLows.drop('YearDay', axis=1, inplace=True)
    highestMonthlyLow = monthlyLows.groupby(monthlyLows.index.month).max(numeric_only=True).round(decimals)
    highestMonthlyLow.index = highestMonthlyLow.index.astype(int)
    # Record years
    highestMonthlyLowYear = monthlyLows.groupby(monthlyLows.index.month).apply(lambda x: x.idxmax(numeric_only=True).dt.year, include_groups=False)
    highestMonthlyLowYear.index = highestMonthlyLowYear.index.astype(int)
    highestMonthlyLowYear.columns = [i+' Year' for i in highestMonthlyLowYear.columns]
    # Create xarray
    results = pd.concat((highestMonthlyLow, highestMonthlyLowYear), axis=1)
    results = xr.DataArray(results, dims=['month', 'variable'])
    results.name = 'Highest Monthly Low'
    return results

def record_daily_low(df, decimals=1):
    """Record daily lows rounded to 'decimals'."""
    # Calculate the record
    dailyLows = daily_lows(df)
    recordLow = dailyLows.groupby('YearDay').min(numeric_only=True).round(decimals)
    recordLow.index = recordLow.index.astype(int)
    # Record years
    recordLowYear = dailyLows.groupby('YearDay').apply(lambda x: x.idxmin(numeric_only=True).dt.year, include_groups=False)
    recordLowYear.index = recordLowYear.index.astype(int)
    recordLowYear.columns = [i+' Year' for i in recordLowYear.columns]
    # Create xarray
    results = pd.concat((recordLow, recordLowYear), axis=1)
    results = xr.DataArray(results, dims=['yearday', 'variable'])
    results.name = 'Record Daily Low'
    return results

def record_monthly_low(df, decimals=1, true_average=False):
    """Record monthly lows rounded to 'decimals'. If 'true_average' is
    True, all measurements from each 24-hour day will be used to calculate
    the daily average. Otherwise, only the maximum and minimum observations
    are used. Defaults to False (meteorological standard).
    """
    # Calculate the record
    monthlyLows = monthly_lows(df, decimals=decimals, true_average=true_average)
    monthlyLows.drop('YearDay', axis=1, inplace=True)
    recordMonthlyLow = monthlyLows.groupby(monthlyLows.index.month).min(numeric_only=True).round(decimals)
    recordMonthlyLow.index = recordMonthlyLow.index.astype(int)
    # Record years
    recordMonthlyLowYear = monthlyLows.groupby(monthlyLows.index.month).apply(lambda x: x.idxmin(numeric_only=True).dt.year, include_groups=False)
    recordMonthlyLowYear.index = recordMonthlyLowYear.index.astype(int)
    recordMonthlyLowYear.columns = [i+' Year' for i in recordMonthlyLowYear.columns]
    # Create xarray
    results = pd.concat((recordMonthlyLow, recordMonthlyLowYear), axis=1)
    results = xr.DataArray(results, dims=['month', 'variable'])
    results.name = 'Record Monthly Low'
    return results

def number_of_years_byday(df):
    """Number of years in the historical data records by day of year."""
    numYears = pd.concat([df[[v, 'YearDay']].dropna().groupby('YearDay')\
                                        .apply(lambda x: len(x.index.year.unique()), include_groups=False) \
                         for v in filtered_data.columns if v != 'YearDay'], axis=1)
    numYears.columns = [v for v in df.columns if v != 'YearDay']
    results = xr.DataArray(numYears, dims=['yearday', 'variable'])
    results.name = 'Number of Years'
    return results

def number_of_years_bymonth(df):
    """Number of years in the historical data records by month."""
    numYears = pd.concat([df[v].dropna().groupby(df[v].dropna().index.month)\
                                        .apply(lambda x: len(x.index.year.unique()), include_groups=False) \
                         for v in filtered_data.columns if v != 'YearDay'], axis=1)
    numYears.columns = [v for v in df.columns if v != 'YearDay']
    results = xr.DataArray(numYears, dims=['yearday', 'variable'])
    results.name = 'Number of Years'
    return results

def generate_yeardays():
    return pd.date_range(start='2020-01-01',end='2020-12-31', freq='1D').strftime('%d-%b')


In [381]:
daily_records = \
    xr.Dataset({'Daily Average': daily_avg(filtered_data),
                'Record High Daily Average': record_high_daily_avg(filtered_data),
                'Record Low Daily Average': record_low_daily_avg(filtered_data),
                'Average High': avg_daily_high(filtered_data),
                'Lowest High': lowest_daily_high(filtered_data),
                'Record High': record_daily_high(filtered_data),
                'Average Low': avg_daily_low(filtered_data),
                'Highest Low': highest_daily_low(filtered_data),
                'Record Low': record_daily_low(filtered_data),
                'Years': number_of_years_byday(filtered_data)},
               attrs={k:v for k, v in meta.items() if k not in ['outdir', 'variables', 'units']})
for k, v in meta['units'].items():
    daily_records.attrs[k+' units'] = v

In [382]:
daily_records

<xarray.Dataset> Size: 120kB
Dimensions:                    (yearday: 366, variable: 4)
Coordinates:
  * yearday                    (yearday) int64 3kB 1 2 3 4 5 ... 363 364 365 366
  * variable                   (variable) object 32B 'Air Temperature' ... 'W...
Data variables:
    Daily Average              (yearday, variable) float64 12kB 71.7 nan ... nan
    Record High Daily Average  (yearday, variable) float64 12kB 78.0 ... 2.02...
    Record Low Daily Average   (yearday, variable) float64 12kB 54.4 ... 2.01...
    Average High               (yearday, variable) float64 12kB 75.2 nan ... nan
    Lowest High                (yearday, variable) float64 12kB 63.3 ... 2.01...
    Record High                (yearday, variable) float64 12kB 79.3 ... 2.02...
    Average Low                (yearday, variable) float64 12kB 68.3 nan ... nan
    Highest Low                (yearday, variable) float64 12kB 76.8 ... 2.02...
    Record Low                 (yearday, variable) float64 12kB 45.5 ... 2.01...
    Years                      (yearday, variable) float64 12kB 22.0 nan ... nan
Attributes:
    stationname:              Virginia Key, FL
    stationid:                8723214
    unit_system:              english
    tz:                       lst
    datum:                    MHHW
    hr_threshold:             3
    day_threshold:            2
    Air Temperature units:    F
    Water Temperature units:  F

In [383]:
monthly_records = \
    xr.Dataset({'Monthly Average': monthly_avg(filtered_data),
                'Record High Monthly Average': record_high_monthly_avg(filtered_data),
                'Record Low Monthly Average': record_low_monthly_avg(filtered_data),
                'Average High': avg_monthly_high(filtered_data),
                'Lowest High': lowest_monthly_high(filtered_data),
                'Record High': record_monthly_high(filtered_data),
                'Average Low': avg_monthly_low(filtered_data),
                'Highest Low': highest_monthly_low(filtered_data),
                'Record Low': record_monthly_low(filtered_data),
                'Years': number_of_years_bymonth(filtered_data)},
               attrs={k:v for k, v in meta.items() if k not in ['outdir', 'variables', 'units']})
for k, v in meta['units'].items():
    monthly_records.attrs[k+' units'] = v

In [384]:
monthly_records

<xarray.Dataset> Size: 4kB
Dimensions:                      (month: 12, variable: 4, yearday: 12)
Coordinates:
  * month                        (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * variable                     (variable) object 32B 'Air Temperature' ... ...
  * yearday                      (yearday) int32 48B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    Monthly Average              (month, variable) float64 384B 68.6 nan ... nan
    Record High Monthly Average  (month, variable) float64 384B 72.6 ... 2.01...
    Record Low Monthly Average   (month, variable) float64 384B 63.0 ... 2.01...
    Average High                 (month, variable) float64 384B 75.9 nan ... nan
    Lowest High                  (month, variable) float64 384B 73.0 ... 2.00...
    Record High                  (month, variable) float64 384B 78.0 ... 2.01...
    Average Low                  (month, variable) float64 384B 55.3 nan ... nan
    Highest Low                  (month, variable) float64 384B 63.5 ... 2.01...
    Record Low                   (month, variable) float64 384B 48.3 ... 2.01...
    Years                        (yearday, variable) float64 384B 22.0 ... nan
Attributes:
    stationname:              Virginia Key, FL
    stationid:                8723214
    unit_system:              english
    tz:                       lst
    datum:                    MHHW
    hr_threshold:             3
    day_threshold:            2
    Air Temperature units:    F
    Water Temperature units:  F

In [322]:
daily_records.sel(variable='Air Temperature').to_dataframe().drop('variable', axis=1)

,Daily Average,Record High Daily Average,Record Low Daily Average,Average High,Lowest High,Record High,Average Low,Highest Low,Record Low,Years
yearday,,,,,,,,,,
1,71.7,78.0,54.4,75.2,63.3,79.3,68.3,76.8,45.5,22.0
2,72.0,77.8,56.6,75.3,64.2,79.3,68.8,76.3,49.1,22.0
3,70.2,78.0,51.7,74.0,57.0,79.2,66.4,77.0,46.4,22.0
4,69.2,77.9,51.6,73.0,58.7,81.7,65.4,76.8,43.7,21.0
5,68.2,77.3,51.0,72.8,57.8,80.2,63.5,75.9,44.2,22.0
...,...,...,...,...,...,...,...,...,...,...
362,67.9,78.4,48.8,72.6,55.4,80.2,63.3,77.5,42.1,23.0
363,69.6,79.0,52.2,73.1,57.7,80.1,66.0,77.9,42.4,23.0
364,70.7,79.4,58.7,74.6,67.1,80.6,66.9,78.3,50.1,23.0


In [386]:
daily_records.to_netcdf(os.path.join(outdir, 'statistics-daily.nc'), mode='w')
monthly_records.to_netcdf(os.path.join(outdir, 'statistics-monthly.nc'), mode='w')

In [387]:
daily_records.equals(daily_records)

True